In [1]:
import torch


NB_AMO=2
CALENDAR_UNITY_HOURS=0.5 # half hour / need to be hour fraction
CALENDAR_DURATION_DAYS=6 # number of days planned
CALENDAR_DURATION_HOURS=30*24 # 
CLOSE_NIGHT_AT=18
OPEN_DAY_AT=10
CLOSE_LUNCH_AT=13
CLOSE_LUNCH_AT=int((CLOSE_LUNCH_AT-OPEN_DAY_AT)/CALENDAR_UNITY_HOURS)
OPEN_LUNCH_AT=12
OPEN_LUNCH_AT=int((OPEN_LUNCH_AT-OPEN_DAY_AT)/CALENDAR_UNITY_HOURS)
OPEN_HOURS = int((CLOSE_NIGHT_AT-OPEN_DAY_AT)/CALENDAR_UNITY_HOURS)
# print(CLOSE_LUNCH_AT,OPEN_LUNCH_AT,OPEN_HOURS)
calendar_model = torch.ones(CALENDAR_DURATION_DAYS,int(OPEN_HOURS), NB_AMO).int() # DAYS (D) x OPEN HOURS (OH) x NB AMO (NA)
calendar = torch.ones(OPEN_HOURS).int() # D 
calendar_mask = torch.ones(OPEN_HOURS).bool() # D 
calendar_mask[OPEN_LUNCH_AT:CLOSE_LUNCH_AT] = 0 # D 
calendar_mask_model = calendar.unsqueeze(1).expand(calendar_model.size()) # D x OH x NA
# print(calendar.size(), calendar_mask.size())
# print(calendar[0,:,1], calendar_mask[2,:,1])

masked_cal = calendar & calendar_mask # D 
# masked_cal[2,:,1]

count=torch.unique_consecutive(masked_cal, return_counts=True, dim=0) # return value and number of consecutives values (2,1,1,2,2,2) -> ((2,1),(1,2),(2,3))
print(count[0].size(), count[1], masked_cal)
fwd_cal = torch.ones(OPEN_HOURS).int() # to be filled with : number of units available until next reserved per slot
bwd_cal = torch.ones(OPEN_HOURS).int() # to be filled with : number of units available since last reserved per slot
last_zero=0
for i in range(calendar.size(0)): # loop days
    if masked_cal[i]!=0:
        val = masked_cal[last_zero:i+1].sum() # count number of units available since last reserved met
#         print(masked_cal[last_zero:i+1], val)
        bwd_cal[i] = val
    else:
        last_zero = i
#         print(last_zero)
        
        bwd_cal[i]=0
        
t_masked_cal = torch.flip(masked_cal,[0])
last_zero=0
for i in range(calendar.size(0)):
    if t_masked_cal[i]!=0:
        val = t_masked_cal[last_zero:i+1].sum() # count number of units available since last reserved met (on reversed caleendar)
#         print(t_masked_cal[last_zero:i+1], val)
        fwd_cal[i] = val
    else:
        last_zero = i
#         print(last_zero)
        fwd_cal[i]=0
        
fwd_cal = torch.flip(fwd_cal,[0])
print(bwd_cal,fwd_cal)
# fwd_cal = 
# masked_cal=

torch.Size([3]) tensor([ 4,  2, 10]) tensor([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)
tensor([ 1,  2,  3,  4,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       dtype=torch.int32) tensor([ 4,  3,  2,  1,  0,  0, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1],
       dtype=torch.int32)


In [2]:
bwd_cal = bwd_cal.unsqueeze(1).expand(calendar_model.size()) # D x OH x NA
fwd_cal = fwd_cal.unsqueeze(1).expand(calendar_model.size()) # D x OH x NA
masked_cal = masked_cal.unsqueeze(1).expand(calendar_model.size()) # D x OH x NA

print(bwd_cal[2,:,1],fwd_cal[2,:,1])
print(bwd_cal.size(),fwd_cal.size(), masked_cal.size())

tensor([ 1,  2,  3,  4,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       dtype=torch.int32) tensor([ 4,  3,  2,  1,  0,  0, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1],
       dtype=torch.int32)
torch.Size([6, 16, 2]) torch.Size([6, 16, 2]) torch.Size([6, 16, 2])


In [3]:
CUSTOMER_DURATIONS_NEED = [(0.5,20),(1,10),(2,10),(3,12),(4,8)]

In [4]:
def get_available(duration, strict = False, equal = False):
#     print(duration, (fwd_cal>= duration).sum().item())
    if equal:
        return (fwd_cal == duration),(fwd_cal == duration).sum().item()
    if strict:
        return (fwd_cal > duration),(fwd_cal > duration).sum().item()
    return (fwd_cal >= duration),(fwd_cal >= duration).sum().item()

In [5]:
get_available(2)[1]
CUSTOMER_DURATION = [t[0]*2 for t in CUSTOMER_DURATIONS_NEED]
CUSTOMER_AVAILABLE = [get_available(t)[1] for t in CUSTOMER_DURATION]
CUSTOMER_NEED = [t[1] for t in CUSTOMER_DURATIONS_NEED]
NEED_AVAIL_RATIO = [round((a)/(n+0.0000001),2) for n,a in zip(CUSTOMER_NEED, CUSTOMER_AVAILABLE) ]
NEED_AVAIL_MATCH = [get_available(t, equal = True)[1] for t in CUSTOMER_DURATION]
NEED_AVAIL_MATCH_PC = [m/a*100 for m,a in zip (NEED_AVAIL_MATCH, CUSTOMER_AVAILABLE)]

In [6]:
print(CUSTOMER_AVAILABLE)
print(CUSTOMER_NEED)
print(NEED_AVAIL_RATIO)
ALL_NEED = [t[0]*t[1]*2 for t in CUSTOMER_DURATIONS_NEED]
print(ALL_NEED, sum(ALL_NEED), CUSTOMER_AVAILABLE[0])

[168, 144, 96, 60, 36]
[20, 10, 10, 12, 8]
[8.4, 14.4, 9.6, 5.0, 4.5]
[20.0, 20, 40, 72, 64] 216.0 168


In [7]:
print([round(t*100/max(NEED_AVAIL_RATIO),2) for t in NEED_AVAIL_RATIO])

[58.33, 100.0, 66.67, 34.72, 31.25]


In [8]:
count=torch.unique_consecutive(masked_cal, return_counts=True, dim=0)
# print(count[0].size(), count[1], masked_cal)
def get_fwd_bwd(ref):
    fwd_cal = torch.empty(OPEN_HOURS).int()
    bwd_cal = torch.empty(OPEN_HOURS).int()
    last_zero=0
    for i in range(ref.size(0)):
        if ref[i]!=0:
            val = ref[last_zero:i+1].sum()
            bwd_cal[i] = val
        else:
            last_zero = i
            bwd_cal[i]=0

    t_masked_cal = torch.flip(ref,[0])
    last_zero=0
    for i in range(ref.size(0)):
        if t_masked_cal[i]!=0:
            val = t_masked_cal[last_zero:i+1].sum()
            fwd_cal[i] = val
        else:
            last_zero = i
            fwd_cal[i]=0

    fwd_cal = torch.flip(fwd_cal,[0])
    print(fwd_cal,bwd_cal)
    return fwd_cal,bwd_cal

In [9]:
def update_calendar_day(day, hour, duration, amo, calendar_model, bwd_cal, fwd_cal):
    print(day,hour, duration)
    START_AT=int((hour-OPEN_DAY_AT)/CALENDAR_UNITY_HOURS)
    END_AT=START_AT+duration*2
    target = torch.ones(calendar_model.size(1))
    target[START_AT:END_AT]=0
    target= calendar_model[day,:,amo] & target.bool()
    print(target)
    print(calendar_model[day,:,amo])
    calendar_model[day,:,amo]=target
    print(calendar_model[day,:,amo])
    fwd_cal_update, bwd_cal_update = get_fwd_bwd(target)
    fwd_cal[day,:,amo]=fwd_cal_update
    bwd_cal[day,:,amo]=bwd_cal_update
    return calendar_model, fwd_cal, bwd_cal
    
    
    
    

In [10]:
day=3
hour=10
duration=2
amo=1
calendar_model, fwd_cal, bwd_cal = update_calendar_day(day, hour, duration, amo, masked_cal, bwd_cal, fwd_cal)

3 10 2
tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)
tensor([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)
tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)
tensor([ 0,  0,  0,  0,  0,  0, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1],
       dtype=torch.int32) tensor([ 0,  0,  0,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       dtype=torch.int32)


In [11]:
print(fwd_cal[day,:,amo], bwd_cal[day,:,amo])

tensor([ 0,  0,  0,  0,  0,  0, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1],
       dtype=torch.int32) tensor([ 0,  0,  0,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       dtype=torch.int32)


In [16]:
def get_lookup_table(max_slot):
    lt = -torch.ones(max_slot, max_slot, max_slot)
    for dcontainer in range(max_slot):
        for dneed in range(max_slot):
            if dneed <= dcontainer:
                for idx in range(dcontainer - dneed + 1):
                    before = int((dcontainer+1)/(dneed+1))
                    after = int((dcontainer+1)/(dcontainer - dneed + idx + 1))
                    
                    lt[dcontainer,dneed,idx] = before + after
                print(dcontainer+1,dneed+1,idx)
                print(before,after)


    return lt

print(get_lookup_table(5))

1 1 0
1 1
2 1 1
2 0
2 2 0
1 2
3 1 2
3 0
3 2 1
1 1
3 3 0
1 3
4 1 3
4 0
4 2 2
2 0
4 3 1
1 1
4 4 0
1 4
5 1 4
5 0
5 2 3
2 0
5 3 2
1 1
5 4 1
1 1
5 5 0
1 5
tensor([[[ 2., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.]],

        [[ 3.,  2., -1., -1., -1.],
         [ 3., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.]],

        [[ 4.,  3.,  3., -1., -1.],
         [ 2.,  2., -1., -1., -1.],
         [ 4., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.]],

        [[ 5.,  4.,  4.,  4., -1.],
         [ 3.,  3.,  2., -1., -1.],
         [ 3.,  2., -1., -1., -1.],
         [ 5., -1., -1., -1., -1.],
         [-1., -1., -1., -1., -1.]],

        [[ 6.,  5.,  5.,  5.,  5.],
         [ 3.,  3.,  2.,  2., -1.],
         [ 2.,  2.,  2., -1., -1.],
         [ 3.,

In [ ]:
def freeze_slot(hot_duration, cold_duration, calendar):
    
    return None